In [0]:

%run "./Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

Created user-specific database

Using the database saifahmed_k_outlook_com_db .

All done!

In [0]:
inputPath = "dbfs:/databricks-datasets/online_retail/data-001/data.csv"
DataPath = userhome + "/delta/customer-data/"

#remove directory if it exists
dbutils.fs.rm(DataPath, True)

Out[9]: True

In [0]:
DataPath

Out[11]: 'dbfs:/user/saifahmed.k@outlook.com/delta/customer-data/'

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

(spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath)
  .write
  .mode("overwrite")
  .format("delta")
  .partitionBy("Country")
  .save(DataPath))

In [0]:
spark.sql("""
  DROP TABLE IF EXISTS customer_data_delta
""")
spark.sql("""
  CREATE TABLE customer_data_delta
  USING DELTA
  LOCATION '{}'
""".format(DataPath))

Out[15]: DataFrame[]

In [0]:
miniDataInputPath = "/mnt/training/online_retail/outdoor-products/outdoor-products-mini.csv"

(spark
  .read
  .option("header", "true")
  .schema(inputSchema)
  .csv(miniDataInputPath)
  .write
  .format("delta")
  .partitionBy("Country")
  .mode("append")
  .save(DataPath)
)

In [0]:
spark.read.format("json").load("/mnt/training/enb/commonfiles/upsert-data.json").createOrReplaceTempView("upsert_data")

In [0]:
%sql
MERGE INTO customer_data_delta
USING upsert_data
ON customer_data_delta.InvoiceNo = upsert_data.InvoiceNo
  AND customer_data_delta.StockCode = upsert_data.StockCode
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *
